In [ ]:
"""Create Project"""
"""Create instance"""
"""Create database inside instance"""
"""create Google Cloud Storage bucket for your project"""
"""Write SQL to create tables and save as txt in your project bucket"""
"""** add data to your SQL to test""" 
"""Install mysql.connector"""
"""https://www.dataquest.io/blog/sql-insert-tutorial/"""
!pip install mysql.connector
!curl ipecho.net/plain  # look at your ip address for your jupyter notebook

In [ ]:
"""Connect to DB"""
"""First add your ip address to authorized networks"""
"""code to get google colab ip address to add to authorized networks"""

# # This may take a couple minutes
import mysql.connector

mydbname = 'whateveryoulike'
gc_ip = "00.000.00.000"
password_root = '123456'
root_db = 'mydb'

my_connect = mysql.connector.connect(
  host=gc_ip,  # change this with your google cloud address
  user="root",
  passwd= password_root, # your password here
  database=root_db # database name
)
####### end of connection details ####
my_cursor = my_connect.cursor()

# my_cursor.execute("SELECT * FROM amzbiz") # table name

# my_result = my_cursor.fetchone()
# while my_result is not None:
#     print(my_result)
#     my_result = my_cursor.fetchone()

my_cursor.execute(f"DROP TABLE {mydbname}")  # clear out database


In [ ]:
# !pip install pymysql
from sqlalchemy import create_engine
import mysql.connector
!pip install pymysql
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import os 
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings("ignore")


my_connect = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(user="root",
                               host=gc_ip,  # your google cloud address
                               pw=password_root,  # your password here
                               db=root_db))  # database name


"""Read in google drive data"""
"""Mount google drive folders to google colab"""
"""if  you did not create the folder, add a shortcut to your drive"""

directory = '/content/drive/My Drive/Your Folder'
folders = ['select folders if you want']  
ogdf = pd.DataFrame()

for f in folders:
    for root,dirs,files in os.walk(f'{directory}/{f}'):
        for file in files:
            if file.endswith('.csv'):
                # print(year, file)
                temp = pd.read_csv(f'{directory}/{f}/{file}')
                temp['date'] = file.replace('.csv', '')
                ogdf = ogdf.append(temp, ignore_index=True)
                
ogdf['date']= pd.to_datetime(ogdf['date'], errors='coerce')  # change your date to a date!
ogdf.sort_values('date', ascending=False, inplace=True)


pricecol = [ 'column names that are prices']  # must be changed to float
percol = ['column names that are percentages'] # must be changed to float

ogdf[pricecol] = ogdf[pricecol].apply(lambda x: x.str.replace('$',''))
ogdf[percol] = ogdf[percol].apply(lambda x: x.str.replace('%',''))


ogdf = ogdf.replace({',': ''} , regex=True)
ogdf[pricecol] = ogdf[pricecol].astype(float)
ogdf = ogdf.fillna(0) 
ogdf[percol] = ogdf[percol].astype(float)*.01  # change to decimal


# First create ogdf :)
ogdf.to_sql(mydbname, con = my_connect, if_exists = 'append', index=False)  # name the database or append if it already exists

display(len(ogdf))
display(ogdf.tail(1))
